In [1]:
import pandas as pd
import numpy as np
import re

In [5]:
df = pd.read_excel('BIEU THUE XNK 2023 HQQN.xlsx', sheet_name="BT2023", header=2, skiprows=range(3,4), converters={'hs':str, 'dashes': str, 'IT': str}).fillna("")

In [7]:
def fix_gov_data(row):
    if str(row['IT']).startswith("="):
        row['IT'] = "'" + str(row['IT'])
    return row

df = df.apply(fix_gov_data, axis=1) 

# Fix heading (ex: 25020000	Pirít sắt chưa nung.)

In [8]:
def insert_row(df, pos, row):
    df2 = pd.DataFrame(np.insert(df.values, pos, values=row, axis=0))
    df2.columns = df.columns
    return df2

In [9]:
# 2502.00.00

def fix_heading(df):
    for index, row in df.iterrows():
        if re.match('^\d{4}0000$', str(row['hs'])) and not row['description'].startswith('-'):
            if row['hs'][:4] in df['hs'].unique():
                df['description'][index] = "- " + row['description']
                df['dashes'][index] = 1
            else:
                new_row = row.copy()
                new_row['hs'] = row['hs'][:4]
                df['description'][index] = "- " + row['description']
                df['dashes'][index] = 1
                return fix_heading(insert_row(df, index, new_row))
        elif str(row['hs']) == '98050000':
            df['dashes'][index] = 1
            if row['hs'][:4] not in df['hs'].unique():
                new_row = row.copy()
                new_row['hs'] = row['hs'][:4]
                new_row['description'] = row['description'].replace('-', '')
                return fix_heading(insert_row(df, index, new_row))
    return df

# 98.08 cos gach -
def fix_hs(df):
    for index, row in df.iterrows():
        if re.match('^\d{2}.\d{2}$', str(row['hs'])) and str(row['description']).startswith('-'):
            df['hs'][index] = str(row['hs']).replace('.', '') + '.00.00'
    return df

def add_dot_to_hs(row):
    hs = row['hs'].replace('.','')
    if len(hs) > 4 and len(hs) < 7:
        row['hs'] = hs[:4] + '.' + hs[4:]
    elif len(hs) >= 7 and len(hs) <= 8:
        row['hs'] = hs[:4] + '.' + hs[4:6] + '.' + hs[6:]
    elif len(hs) == 4:
        row['hs'] = hs[:2] + '.' + hs[2:]
    return row

def add_dot_to_related_hs(row):
    hs = row['related_hs'].replace('.','')
    if len(hs) > 4 and len(hs) < 7:
        row['related_hs'] = hs[:4] + '.' + hs[4:]
    elif len(hs) >= 7 and len(hs) <= 8:
        row['related_hs'] = hs[:4] + '.' + hs[4:6] + '.' + hs[6:]
    elif len(hs) == 4:
        row['related_hs'] = hs[:2] + '.' + hs[2:]
    return row


fixed_heading = fix_heading(df)
fixed_heading = fixed_heading.apply(add_dot_to_hs, axis=1)
fixed_heading = fixed_heading.apply(add_dot_to_related_hs, axis=1)
fixed_heading = fix_hs(fixed_heading)
# fixed_heading.to_excel('fixed_heading.xlsx', index=False)

# Remove section, chapter, all notes, general

In [10]:
def removeStuff(fixed_heading):
    remove_index = []
    for index, row in fixed_heading.iterrows():
        if not row['hs'] and not row['description'].startswith('- '):
            remove_index.append(index)
    return fixed_heading.drop(remove_index)

removed_df = removeStuff(fixed_heading).reset_index(drop=True)

In [11]:
removed_df["id"] = removed_df.index + 1
cols = removed_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
removed_df[cols].to_excel('removed_df.xlsx', index=False)